In [1]:
import torch
from torch import nn

In [2]:
import torchvision
from torchvision import datasets

In [3]:
from torchvision import transforms

## Fashion MNIST

In [4]:
train_data = datasets.FashionMNIST(root="data", train=True, download=True, target_transform=None,
                                   transform=torchvision.transforms.ToTensor())

test_data = datasets.FashionMNIST(root="data", train=False, download=True, target_transform=None,
                                   transform=torchvision.transforms.ToTensor())

In [5]:
len(train_data), len(test_data)

(60000, 10000)

In [6]:
image, label = train_data[0]

In [7]:
class_names = train_data.classes
class_names

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [8]:
class_to_idx = train_data.class_to_idx

## Visualizing Data

In [9]:
import plotly.express as px
import numpy as np

In [10]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [11]:
from typing import Union

In [12]:
def showImage(image: Union[torch.Tensor, np.array], label: int) -> None:
    """Show Image of Fashion MNIST set with Label

    :param image: Image data
    :type image: Union[torch.Tensor, np.array]

    :param label: Label id
    :type label: int
    """

    class_nm = ['T-shirt/top','Trouser','Pullover','Dress',
                'Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

    if image.ndim == 3:
        image = image.squeeze()

    fig = px.imshow(image, color_continuous_scale='gray', labels=dict(x=f"{class_nm[label]}"))
    fig.update_layout(coloraxis_showscale=False)
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)

    fig.show()

In [13]:
showImage(image, label)

In [14]:
def plotRandomImg(train_data: torchvision.datasets.mnist.FashionMNIST, grid_len=5, seed=42):
    """_summary_

    :param train_data: Total Training Data
    :type train_data: torchvision.datasets.mnist.FashionMNIST

    :param grid_len: Number of grid, defaults to 5. Tot images = grid_len * grid_len
    :type grid_len: int, optional
    """
    torch.manual_seed(seed=seed)

    rand_indexs = torch.randint(0, len(train_data), size=[grid_len*grid_len])

    fig = make_subplots(grid_len, grid_len)
    
    row_ind_start = 1

    class_nm = ['T-shirt/top','Trouser','Pullover','Dress',
                'Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']
    samp_ind = 0

    while row_ind_start <=grid_len:
        col_ind_start = 1
        while col_ind_start <= grid_len:
            img_data, label = train_data[rand_indexs[samp_ind]]
            single_fig = go.Figure(go.Heatmap(z=img_data.squeeze()))
            fig.add_trace(single_fig.data[0], row_ind_start, col_ind_start)
            fig.update_yaxes(autorange="reversed")
            fig.update_xaxes(title_text = f"{class_nm[label]}", row=row_ind_start, col=col_ind_start)
            col_ind_start +=1
            samp_ind += 1
        row_ind_start += 1

    fig.update_layout(height=1000, width=1200)
    fig.update_layout(coloraxis_showscale=False)

    return fig

In [15]:
plotRandomImg(train_data, seed=2122)

In [16]:
from torch.utils.data import DataLoader

In [17]:
BATCH_SIZE = 32

In [18]:
# Turn datasets into iterables (batches)
train_dataloader = DataLoader(train_data, # dataset to turn into iterable
    batch_size=BATCH_SIZE, # how many samples per batch? 
    shuffle=True # shuffle data every epoch?
)

test_dataloader = DataLoader(test_data,
    batch_size=BATCH_SIZE,
    shuffle=False # don't necessarily have to shuffle the testing data
)


In [19]:
train_features_batch, train_labels_batch = next(iter(train_dataloader))

In [20]:
x = train_features_batch[0]

In [21]:
train_features_batch[0].shape

torch.Size([1, 28, 28])

In [22]:
from torch import nn

In [23]:
# Import accuracy metric
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc


In [24]:
# Setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss() # this is also called "criterion"/"cost function" in some places

## Coding for Training & Testing

In [25]:
from tqdm.auto import tqdm

/Users/niteshkumarsharma/Desktop/AI/PyTorch/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [26]:
device = "mps"

In [27]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device):
    train_loss, train_acc = 0, 0
    model.to(device)
    for batch, (X, y) in enumerate(data_loader):
        # Send data to device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")



In [28]:
def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device = device):
    test_loss, test_acc = 0, 0
    model.to(device)
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode(): 
        for X, y in data_loader:
            # Send data to GPU
            X, y = X.to(device), y.to(device)
            
            # 1. Forward pass
            test_pred = model(X)
            
            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )
        
        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [29]:
# Move values to device
torch.manual_seed(42)
def eval_model(model: torch.nn.Module, 
               data_loader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               accuracy_fn, 
               device: torch.device = device):
    """Evaluates a given model on a given dataset.

    Args:
        model (torch.nn.Module): A PyTorch model capable of making predictions on data_loader.
        data_loader (torch.utils.data.DataLoader): The target dataset to predict on.
        loss_fn (torch.nn.Module): The loss function of model.
        accuracy_fn: An accuracy function to compare the models predictions to the truth labels.
        device (str, optional): Target device to compute on. Defaults to device.

    Returns:
        (dict): Results of model making predictions on data_loader.
    """
    loss, acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in data_loader:
            # Send data to the target device
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss += loss_fn(y_pred, y)
            acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))
        
        # Scale loss and acc
        loss /= len(data_loader)
        acc /= len(data_loader)
    return {"model_name": model.__class__.__name__, # only works when model was created with a class
            "model_loss": loss.item(),
            "model_acc": acc}

## 1st CNN Model

In [30]:
class FashionMnistModelV2(nn.Module):
    """Tiny VGG Implmentation in PyTorch
    """

    def __init__(self, input_shape: int, hidden_units:int, output_shape:int):
        super(FashionMnistModelV2, self).__init__()

        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)          
        )

        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.classifer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=490,
                      out_features=output_shape)
        )

    def forward(self, x:torch.tensor):
        x = self.conv_block_1(x)
        # print(x.shape)

        x = self.conv_block_2(x)
        # print(x.shape)

        x = self.classifer(x)
        # print(x.shape)

        return x

In [31]:
torch.manual_seed(34)
model_cnn = FashionMnistModelV2(input_shape=1, hidden_units=10, output_shape=10)

In [32]:
random_img = torch.randn(size=(32,1,28,28))

In [36]:
model_cnn.eval()
with torch.inference_mode():
    model_cnn(random_img[0].unsqueeze(dim=0))

In [37]:
from torchsummary import summary

In [40]:
summary(model_cnn, input_size=(1, 28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
              ReLU-2           [-1, 10, 28, 28]               0
            Conv2d-3           [-1, 10, 28, 28]             910
              ReLU-4           [-1, 10, 28, 28]               0
         MaxPool2d-5           [-1, 10, 14, 14]               0
            Conv2d-6           [-1, 10, 14, 14]             910
              ReLU-7           [-1, 10, 14, 14]               0
            Conv2d-8           [-1, 10, 14, 14]             910
              ReLU-9           [-1, 10, 14, 14]               0
        MaxPool2d-10             [-1, 10, 7, 7]               0
          Flatten-11                  [-1, 490]               0
           Linear-12                   [-1, 10]           4,910
Total params: 7,740
Trainable params: 7,740
Non-trainable params: 0
-----------------------------------

In [41]:
optimizer = torch.optim.SGD(params=model_cnn.parameters(), lr=0.1)

In [42]:
epochs = 10
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train_step(data_loader=train_dataloader, 
        model=model_cnn, 
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn,
        device="mps"
    )
    test_step(data_loader=test_dataloader,
        model=model_cnn,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn,
        device="mps")

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0
---------
Train loss: 0.60468 | Train accuracy: 78.13%


 10%|█         | 1/10 [00:08<01:17,  8.57s/it]

Test loss: 0.41118 | Test accuracy: 85.14%

Epoch: 1
---------
Train loss: 0.35370 | Train accuracy: 87.24%


 20%|██        | 2/10 [00:16<01:07,  8.42s/it]

...

Test loss: 0.29465 | Test accuracy: 89.36%

Epoch: 8
---------
Train loss: 0.25176 | Train accuracy: 90.85%


 90%|█████████ | 9/10 [01:15<00:08,  8.34s/it]

Test loss: 0.28725 | Test accuracy: 89.34%

Epoch: 9
---------
Train loss: 0.24770 | Train accuracy: 90.89%


100%|██████████| 10/10 [01:23<00:00,  8.37s/it]

Test loss: 0.29636 | Test accuracy: 89.47%



In [ ]:
device

In [ ]:
model_cnn_results = eval_model(model=model_cnn, data_loader=test_dataloader,
                               loss_fn=loss_fn,
                               accuracy_fn=accuracy_fn,
                               device=device)

In [ ]:
model_cnn_results

In [ ]:
# model_cnn.eval()
# with torch.inference_mode():
#     model_cnn(train_features_batch[0].to(device))

In [ ]:
# train_features_batch.shape

In [ ]:
def make_predictions(model: nn.Module, data: list, device: torch.device = device):
    """Make & Evaluate Random Predictions with the Best Model

    :param model: Train CNN Model
    :type model: nn.Module

    :param data: List of Input Tensors
    :type data: list

    :param device: _description_, defaults to device
    :type device: torch.device, optional
    """

    pred_probs = []

    # Setting Model on Eval Mode #
    model.eval()

    with torch.inference_mode():
        for sample in data:
            # Prepare Sample #
            sample = torch.unsqueeze(sample, dim=0).to(device)

            # Model Output(This would be raw logits)
            pred_logit = model(sample)

            # Get Prediction Probability (logit -> prediction Probabilty)
            pred_prob = torch.softmax(pred_logit.squeeze(), dim=0)

            # Get Pred_Prob off GPU for furthure Calculations #
            pred_probs.append(pred_prob.cpu())

    return torch.stack(pred_probs)

In [ ]:
import random
random.seed(42)

In [ ]:
test_samples = []
test_labels = []

for sample, label in random.sample(list(test_data), k = 9):
    test_samples.append(sample)
    test_labels.append(label)

In [ ]:
print(f"Test sample image shape: {test_samples[0].shape}\nTest sample label: {test_labels[0]} ({class_names[test_labels[0]]})")

In [ ]:
pred_probs = make_predictions(model=model_cnn, data=test_samples)

In [ ]:
pred_probs.shape

In [ ]:
pred_classes = pred_probs.argmax(dim=1)

In [ ]:
test_labels, pred_classes

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Plot predictions
plt.figure(figsize=(9, 9))
nrows = 3
ncols = 3
for i, sample in enumerate(test_samples):
  # Create a subplot
  plt.subplot(nrows, ncols, i+1)

  # Plot the target image
  plt.imshow(sample.squeeze(), cmap="gray")

  # Find the prediction label (in text form, e.g. "Sandal")
  pred_label = class_names[pred_classes[i]]

  # Get the truth label (in text form, e.g. "T-shirt")
  truth_label = class_names[test_labels[i]] 

  # Create the title text of the plot
  title_text = f"Pred: {pred_label} | Truth: {truth_label}"
  
  # Check for equality and change title colour accordingly
  if pred_label == truth_label:
      plt.title(title_text, fontsize=10, c="g") # green text if correct
  else:
      plt.title(title_text, fontsize=10, c="r") # red text if wrong
  plt.axis(False);


## Plotting Confusion Matrix!

In [ ]:
y_preds = []
model_cnn.eval()
with torch.inference_mode():
    for X, y in tqdm(test_dataloader, desc="Making Preds"):
        X, y = X.to(device), y.to(device)

        y_logit = model_cnn(X)

        y_pred = torch.softmax(y_logit, dim=1).argmax(dim=1)

        y_preds.append(y_pred.cpu())

In [ ]:
y_pred_tensor = torch.cat(y_preds)

In [ ]:
X.shape

In [ ]:
train_features_batch.shape

In [ ]:
model_cnn.eval()
with torch.inference_mode():
     model_cnn(X)

In [ ]:
import torchmetrics

In [ ]:
y_pred_tensor.shape

In [ ]:
test_data.targets.shape

In [ ]:
bool_tensor = y_pred_tensor == test_data.targets

In [ ]:
bool_tensor

In [ ]:
~bool_tensor

## Analysis Where Model is Performing Very Poorly


In [ ]:
test_data_wrng_pred = test_data.data[~bool_tensor]
test_wrong_gt = test_data.targets[~bool_tensor]

In [ ]:
test_data_wrng_pred.shape

In [ ]:
test_data_wrng_pred = test_data_wrng_pred.float()

In [ ]:
test_data_wrng_pred.device

In [ ]:
model_cnn.eval()
with torch.inference_mode():
    test_data_wrng_pred = test_data_wrng_pred.to(device)
    test_data_rt_shape = test_data_wrng_pred.unsqueeze(dim=1)
    preds_logits = model_cnn(test_data_rt_shape)

    preds_prob = torch.softmax(preds_logits, dim=1)

    preds_label = preds_prob.argmax(dim=1)

    

In [ ]:
# model_cnn.eval()
# with torch.inference_mode():
#     chk = model_cnn(test_data_rt_shape[555])

In [ ]:
preds_label

In [ ]:
test_wrong_gt[idx]

In [ ]:
class_names

In [ ]:
idx = 555

In [ ]:
single_input = test_data_wrng_pred[idx].unsqueeze(dim=0)
single_input_ = single_input.unsqueeze(dim=0)

In [ ]:
# model_cnn.eval()
# with torch.inference_mode():
#     logit_pred = model_cnn(single_input)

In [ ]:
logit_pred_prob = torch.softmax(logit_pred, dim=1)

In [ ]:
logit_pred_prob

In [ ]:
test_wrong_gt[idx]

In [ ]:
showImage(test_data_wrng_pred[idx].cpu(), test_wrong_gt[idx])
print(class_names[test_wrong_gt[idx]])

In [ ]:
import mlxtend

In [ ]:
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

# 2. Setup confusion matrix instance and compare predictions to targets
confmat = ConfusionMatrix(num_classes=len(class_names), task='multiclass')
confmat_tensor = confmat(preds=y_pred_tensor, target=test_data.targets)

# 3. Plot the confusion matrix
fig, ax = plot_confusion_matrix(
    conf_mat=confmat_tensor.numpy(), # matplotlib likes working with NumPy 
    class_names=class_names, # turn the row and column labels into class names
    figsize=(10, 7)
);